In [5]:
# Create the directory if it doesn't exist
!mkdir ..\data

# Download files using curl
!curl -o ../data/green_tripdata_2024-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet
!curl -o ../data/green_tripdata_2024-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 87 1330k   87 1167k    0     0  3115k      0 --:--:-- --:--:-- --:--:-- 3129k
100 1330k  100 1330k    0     0  3363k      0 --:--:-- --:--:-- --:--:-- 3385k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1253k  100 1253k    0     0  2195k      0 --:--:-- --:--:-- --:--:-- 2207k


In [6]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [7]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [8]:
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

In [9]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [10]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [11]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
import dagshub
import mlflow


dagshub.init(url="https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=de7bda49-ee66-494d-8db4-1d507f16e616&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=573fcd3653c61671932928678e2b4e23a631511af750e32ff46eaa5c8eaead94




Output()

Accessing as PacoTinoco

Initialized MLflow to track repo "PacoTinoco/nyc-taxi-time-prediction"

Repository PacoTinoco/nyc-taxi-time-prediction initialized!

https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow


2024/09/17 21:28:12 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/9e991ac0af45417cb6a8c46fbaf1900e', creation_time=1726630091495, experiment_id='0', last_update_time=1726630091495, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [14]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")

In [16]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("PacoTinoco/nyc-taxi-time-prediction")

# Upload file
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-01.parquet",  # local path of file to upload
    Key="train_data.parquet",  # remote path where to upload the file
)

## test
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-02.parquet",  # local path of file to upload
    Key="eval_data.parquet",  # remote path where to upload the file
)


Client created. Use the name of the repo (nyc-taxi-time-prediction) as the name of the bucket

In [17]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import pathlib

In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    with mlflow.start_run(nested=True):
         
        # Tag model
        mlflow.set_tag("model_family", "xgboost")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        
        # Log xgboost model with artifact_path
        mlflow.xgboost.log_model(booster, artifact_path="model")
         
        # Predict in the val dataset
        y_pred = booster.predict(valid)
        
        # Calculate metric
        rmse = root_mean_squared_error(y_val, y_pred)
        
        # Log performance metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [20]:
mlflow.xgboost.autolog()

with mlflow.start_run(run_name="Xgboost Hyper-parameter Optimization", nested=True):
    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"
    
    mlflow.log_params(best_params)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "NYC Taxi Time Prediction Project",
            "optimizer_engine": "hyper-opt",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )
        
    y_pred = booster.predict(valid)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    pathlib.Path("models").mkdir(exist_ok=True)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

[0]	validation-rmse:5.74156                                                                                                                                     
[1]	validation-rmse:5.45989                                                                                                                                     
[2]	validation-rmse:5.42515                                                                                                                                     
[3]	validation-rmse:5.41843                                                                                                                                     
[4]	validation-rmse:5.38819                                                                                                                                     
[5]	validation-rmse:5.38806                                                                                                                                     
[6]	validation-rmse:5.37141       

2024/09/17 21:38:57 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/17 21:38:57 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:38:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/17 21:39:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-pr

[0]	validation-rmse:8.51621                                                                                                                                     
[1]	validation-rmse:8.00305                                                                                                                                     
[2]	validation-rmse:7.56157                                                                                                                                     
[3]	validation-rmse:7.18444                                                                                                                                     
[4]	validation-rmse:6.86322                                                                                                                                     
[5]	validation-rmse:6.59146                                                                                                                                     
[6]	validation-rmse:6.36126       

2024/09/17 21:39:58 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/17 21:39:58 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:39:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/17 21:40:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-pr

[0]	validation-rmse:6.42191                                                                                                                                     
[1]	validation-rmse:5.57645                                                                                                                                     
[2]	validation-rmse:5.34412                                                                                                                                     
[3]	validation-rmse:5.26298                                                                                                                                     
[4]	validation-rmse:5.23162                                                                                                                                     
[5]	validation-rmse:5.22004                                                                                                                                     
[6]	validation-rmse:5.21589       

...

2024/09/17 21:43:19 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/17 21:43:19 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:43:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/17 21:43:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-pr

[0]	validation-rmse:8.25705                                                                                                                                     
[1]	validation-rmse:7.58358                                                                                                                                     
[2]	validation-rmse:7.06088                                                                                                                                     
[3]	validation-rmse:6.63837                                                                                                                                     
[4]	validation-rmse:6.32789                                                                                                                                     
[5]	validation-rmse:6.07835                                                                                                                                     
[6]	validation-rmse:5.90129       

2024/09/17 21:44:29 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/17 21:44:29 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:44:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/17 21:44:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-pr

100%|██████████████████████████████████████████████████████████████████████████████████████████| 10/10 [06:12<00:00, 37.22s/trial, best loss: 5.161312656421218]
[0]	validation-rmse:8.25745
[1]	validation-rmse:7.57626
[2]	validation-rmse:7.03381
[3]	validation-rmse:6.61336
[4]	validation-rmse:6.28612
[5]	validation-rmse:6.03057
[6]	validation-rmse:5.83800
[7]	validation-rmse:5.69478
[8]	validation-rmse:5.58245
[9]	validation-rmse:5.49834
[10]	validation-rmse:5.43310
[11]	validation-rmse:5.38299
[12]	validation-rmse:5.34679
[13]	validation-rmse:5.31251
[14]	validation-rmse:5.28809
[15]	validation-rmse:5.26754
[16]	validation-rmse:5.25316
[17]	validation-rmse:5.24021
[18]	validation-rmse:5.22815
[19]	validation-rmse:5.21918
[20]	validation-rmse:5.21247
[21]	validation-rmse:5.20694
[22]	validation-rmse:5.20071
[23]	validation-rmse:5.19726
[24]	validation-rmse:5.19349
[25]	validation-rmse:5.19030
[26]	validation-rmse:5.18899
[27]	validation-rmse:5.18694
[28]	validation-rmse:5.18703
[29]	val

2024/09/17 21:45:19 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/09/17 21:45:19 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:45:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2024/09/17 21:45:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/17 21:45:28 INFO mlflow.tracking._tracking_service.client: 🏃 Vie

In [21]:
best_params

{'learning_rate': np.float64(0.14599396824069683),
 'max_depth': 42,
 'min_child_weight': np.float64(1.7855145549554956),
 'reg_alpha': np.float64(0.05020597797620721),
 'reg_lambda': np.float64(0.0036879070748615603),
 'seed': 42,
 'objective': 'reg:squarederror'}

In [22]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Ingrese el run_id 23b66a83aacb4d04a076a7275dfc65e3


Successfully registered model 'nyc-taxi-model'.
2024/09/17 21:51:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-model, version 1
Created version '1' of model 'nyc-taxi-model'.


In [23]:
from datetime import datetime
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="nyc-taxi-model",
    description="Model registry for the NYC Taxi Time Prediction Project",
)

new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model "nyc-taxi-model"
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="nyc-taxi-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=['champion'], creation_timestamp=1726631483611, current_stage='None', description='The model version 1 was transitioned to champion on 2024-09-17 21:51:28.695308', last_updated_timestamp=1726631487852, name='nyc-taxi-model', run_id='23b66a83aacb4d04a076a7275dfc65e3', run_link='', source='mlflow-artifacts:/9e991ac0af45417cb6a8c46fbaf1900e/23b66a83aacb4d04a076a7275dfc65e3/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [24]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

array([17.868977 , 27.244492 ,  7.5599394, ..., 36.802193 , 10.982415 ,
       19.203161 ], dtype=float32)